# The Battle of the Neighborhoods - Week 2

## Part 1 Download and Explore New York city geographical coordinates dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

First, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Libraries imported.


**The json file is downloaded and it is placed on the server. So run a wget command and access the data.**

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


### Load and explore the data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

**All the relevant data is in the features key, which is basically a list of the neighborhoods. So, define a new variable that includes this data.**

In [4]:
neighborhoods_data = newyork_data['features']

**Take a look at the first item in this list.**

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the data into a pandas dataframe

**The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. Start by creating an empty dataframe.**

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,Borough,Neighborhood,Latitude,Longitude


**Then loop through the data and fill the dataframe one row at a time.**

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


**Let's make sure that the dataset has all 5 boroughs and 306 neighborhoods.**

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


**The dataframe has 5 boroughs and 306 neighborhoods.**

In [10]:
neighborhoods.to_csv('BON1_NYC_GEO.csv',index=False)

### Use geopy library to get the latitude and longitude values of New York City.

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### **Create a map of New York with neighborhoods superimposed on top.**

**Folium is a great visualization library. We can zoom into the below map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.**

In [12]:
# create map of Toronto using latitude and longitude values
map_NewYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NewYork)  
    
map_NewYork

## Part 2 Web scrapping of Population and Demographics data of New York city from Wikipedia

**A : POPULATION DATA
Web scrapping of Population data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City**

In [13]:
from bs4 import BeautifulSoup # package for parsing HTML and XML documents

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Libraries imported.


### **Web scrapping of Population data from wikipedia page using BeautifulSoup.**

Beautiful Soup is a Python package for parsing HTML and XML documents (including having malformed markup, i.e. non-closed tags, so named after tag soup). It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [20]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('BON2_POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Demographics of New York City - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsCFegpAMNQAAtqHKCgAAADA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Demographics_of_New_York_City","wgTitle":"Demographics of New York City","wgCurRevisionId":955864680,"wgRevisionId":955864680,"wgArticleId":1729017,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","CS1 maint: archived copy as title","All articles with dead external links","Art

### Load data from CSV

In [29]:
Pop_data=pd.read_csv('BON2_POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[7,8,9,10,11]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


**Remove whitespaces and rename columns**

In [30]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte,Jurisdiction,Population,GrossDomesticProduct,Landarea,Density,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Landarea\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)
Pop_data

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Replace newline('\n') from each string from left and right sides**

In [26]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2017']=Pop_data['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_miles']=Pop_data['square_miles'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_km']=Pop_data['square_km'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)
#Pop_data['persons_sq_km']=Pop_data['persons_sq_km'].replace(to_replace='\n', value='', regex=True)
Pop_data

,Borough,County,Estimate_2017,GrossDomesticProduct,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,NaN,NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,NaN,NaN,NaN
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,NaN,NaN,NaN
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,NaN,NaN,NaN
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Remove 'NAN'**

In [41]:
Pop_data = Pop_data.fillna('')
Pop_data

,NewYorkCitysfiveboroughsvte,Jurisdiction,Population,GrossDomesticProduct,Landarea,Density,persons_sq_mi,squarekm,persons/sq.mi,persons/km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,,,
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,,,
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,,,
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,,,
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,,,
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",,,
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,,,
7,Sources:[14] and see individual borough articl...,,,,,,,,,


**Save dataframe as csv file**

In [43]:
Pop_data.to_csv('BON2_POPULATION.csv',index=False)

**B : DEMOGRAPHICS DATA**

We will web scrap Demographics data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

Web scrapping of Demographics data from wikipedia page using BeautifulSoup.
Beautiful Soup is a Python package for parsing HTML and XML documents (including having malformed markup, i.e. non-closed tags, so named after tag soup). It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [46]:
website_url = requests.get('https://en.wikipedia.org/wiki/New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable collapsible'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('NYC_DEMO.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

**Load data from CSV**

In [47]:
Demo_data=pd.read_csv('NYC_DEMO.csv')
print('Data downloaded!')

Data downloaded!


In [48]:
Demo_data

,New York City's five boroughsvte,Jurisdiction,Population,Gross Domestic Product,Land area,Density,Borough,County,Estimate (2019)[12],billions(US$)[13],per capita(US$),square miles,squarekm,persons / sq. mi,persons /km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,"33,867\n","13,006\n",NaN,NaN,NaN,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,"36,147\n","13,957\n",NaN,NaN,NaN,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,"71,341\n","27,544\n",NaN,NaN,NaN,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,"20,767\n","8,018\n",NaN,NaN,NaN,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,"8,157\n","3,150\n",NaN,NaN,NaN,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547","10,636\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,159\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Remove whitespaces and rename columns**

In [50]:
Demo_data.columns

Index(['New York City's five boroughsvte\n', 'Jurisdiction\n', 'Population\n',
       'Gross Domestic Product\n', 'Land area\n', 'Density\n', 'Borough',
       'County', 'Estimate (2019)[12]', 'billions(US$)[13]', 'per capita(US$)',
       'square miles', 'squarekm', 'persons / sq. mi', 'persons /km2\n'],
      dtype='object')

In [51]:
Demo_data.rename(columns = {'2010[237]' : '2010',
                   '1990[239]':'1990',
                   '1970[239]':'1970', 
                   '1940[239]\n':'1940',
                    }, inplace=True)
Demo_data

,New York City's five boroughsvte,Jurisdiction,Population,Gross Domestic Product,Land area,Density,Borough,County,Estimate (2019)[12],billions(US$)[13],per capita(US$),square miles,squarekm,persons / sq. mi,persons /km2
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,"33,867\n","13,006\n",NaN,NaN,NaN,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,"36,147\n","13,957\n",NaN,NaN,NaN,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,"71,341\n","27,544\n",NaN,NaN,NaN,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,"20,767\n","8,018\n",NaN,NaN,NaN,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,"8,157\n","3,150\n",NaN,NaN,NaN,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547","10,636\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,159\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
Demo_data.columns

Index(['New York City's five boroughsvte\n', 'Jurisdiction\n', 'Population\n',
       'Gross Domestic Product\n', 'Land area\n', 'Density\n', 'Borough',
       'County', 'Estimate (2019)[12]', 'billions(US$)[13]', 'per capita(US$)',
       'square miles', 'squarekm', 'persons / sq. mi', 'persons /km2\n'],
      dtype='object')

In [53]:
Demo_data.columns = Demo_data.columns.str.replace(' ', '')

**Replace newline('\n') from each string from left and right sides**

In [56]:
Demo_data= Demo_data.replace('\n',' ', regex=True)
Demo_data.drop(['billions(US$)[13]', 'percapita(US$)', 'squaremiles', 'squarekm', 'persons/sq.mi'], axis=1)
Demo_data

,NewYorkCity'sfiveboroughsvte,Jurisdiction,Population,GrossDomesticProduct,Landarea,Density,Borough,County,Estimate(2019)[12],billions(US$)[13],percapita(US$),squaremiles,squarekm,persons/sq.mi,persons/km2
0,The Bronx,Bronx,"1,418,207",42.695,"30,100",42.10,109.04,"33,867","13,006",NaN,NaN,NaN,NaN,NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559,"35,800",70.82,183.42,"36,147","13,957",NaN,NaN,NaN,NaN,NaN,NaN
2,Manhattan,New York,"1,628,706",600.244,"368,500",22.83,59.13,"71,341","27,544",NaN,NaN,NaN,NaN,NaN,NaN
3,Queens,Queens,"2,253,858",93.310,"41,400",108.53,281.09,"20,767","8,018",NaN,NaN,NaN,NaN,NaN,NaN
4,Staten Island,Richmond,"476,143",14.514,"30,500",58.37,151.18,"8,157","3,150",NaN,NaN,NaN,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547","10,636",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Save dataframe as csv file**

In [58]:
Demo_data.to_csv('BON2_DEMOGRAPHICS.csv',index=False)

## Part 3 Download and Explore New York city and its Boroughs Cuisine dataset

In [ ]:
from PIL import Image # converting images into arrays

#!conda install -c conda-forge wordcloud==1.4.1 --yes
# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

In [62]:
# Fetch the file
my_file = project.get_file("BON3_NYC_CUISINE.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
import pandas as pd
NYC_CUISINE=pd.read_csv(my_file)
NYC_CUISINE.drop(NYC_CUISINE.columns[[3,4,5,6,7]], axis=1,inplace=True) 
NYC_CUISINE.head()

NameError: name 'project' is not defined

In [63]:
NYC_CUISINE.shape

NameError: name 'NYC_CUISINE' is not defined

In [ ]:
print(NYC_CUISINE.Borough.unique())

In [ ]:
NYC_CUISINE['Borough'].value_counts().to_frame()

### 1. NEW YORK CITY CUISINE - WORD CLOUD

In [ ]:
CUISINE_WC = NYC_CUISINE[['Cuisine']]
CUISINE_WC

In [ ]:
CUISINE_WC.to_csv('CUISINE_WC.txt', sep=',', index=False)

In [ ]:
CUISINE_WC1 = open('CUISINE_WC.txt', 'r').read()

**Use the stopwords that we imported from word_cloud. We use the function set to remove any redundant stopwords.**

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
# instantiate a word cloud object
NYC_CUISINE_WC = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
NYC_CUISINE_WC.generate(CUISINE_WC1)

**The word cloud is created, let's visualize it.**

In [ ]:
# display the word cloud
plt.imshow(NYC_CUISINE_WC, interpolation='bilinear')
plt.axis('off')

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(45)

plt.show()

**Most Preferred Food in New York City -**

1. Italian
2. Purto Rican
3. Mexican
4. Jewish
5. Indian
6. Pakistani
7. Dominican

### BROOKLYN CUISINE - WORD CLOUD

In [ ]:
Brooklyn_data = NYC_CUISINE[NYC_CUISINE['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

In [ ]:
BR_CUISINE_WC = Brooklyn_data[['Cuisine']]
BR_CUISINE_WC

In [ ]:
BR_CUISINE_WC.to_csv('BR_CUISINE.txt', sep=',', index=False)

In [ ]:
BR_CUISINE_WC = open('BR_CUISINE.txt', 'r').read()

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
# instantiate a word cloud object
BR_CUISINE_NYC = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
BR_CUISINE_NYC.generate(BR_CUISINE_WC)

In [ ]:
# display the word cloud
plt.imshow(BR_CUISINE_NYC, interpolation='bilinear')
plt.axis('off')

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(45)

plt.show()

**Most Preferred Food in Brooklyn is -**

1. Italian
2. Purto Rican
3. Mexican

### QUEENS CUISINE - WORD CLOUD

In [ ]:
Queens_data = NYC_CUISINE[NYC_CUISINE['Borough'] == 'Queens'].reset_index(drop=True)
Queens_data.head()

In [ ]:
Q_CUISINE_WC = Queens_data[['Cuisine']]
Q_CUISINE_WC

In [ ]:
Q_CUISINE_WC.to_csv('Q_CUISINE.txt', sep=',', index=False)

In [ ]:
Q_CUISINE_WC.to_csv('Q_CUISINE.txt', sep=',', index=False)

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
# instantiate a word cloud object
Q_CUISINE_NYC = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
Q_CUISINE_NYC.generate(Q_CUISINE_WC)

In [ ]:
# display the word cloud
plt.imshow(Q_CUISINE_NYC, interpolation='bilinear')
plt.axis('off')

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(45)

plt.show()

**Most Preferred Food in Queens is -**

1. Indian
2. Irish
3. Pakistani
4. Mexican

### MANHATTAN CUISINE - WORD CLOUD

In [ ]:
Manhattan_data = NYC_CUISINE[NYC_CUISINE['Borough'] == 'Manhattan'].reset_index(drop=True)
Manhattan_data.head()

In [ ]:
MN_CUISINE_WC = Manhattan_data[['Cuisine']]
MN_CUISINE_WC

In [ ]:
MN_CUISINE_WC.to_csv('MN_CUISINE.txt', sep=',', index=False)

In [ ]:
MN_CUISINE_WC = open('MN_CUISINE.txt', 'r').read()

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
# instantiate a word cloud object
MN_CUISINE_NYC = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
MN_CUISINE_NYC.generate(MN_CUISINE_WC)

In [ ]:
# display the word cloud
plt.imshow(MN_CUISINE_NYC, interpolation='bilinear')
plt.axis('off')

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(45)

plt.show()

**Most Preferred Food in Manhattan is -**

1. Italian
2. American
3. Puerto Rican
4. Indian

### THE BRONX CUISINE - WORD CLOUD

In [ ]:
Bronx_data = NYC_CUISINE[NYC_CUISINE['Borough'] == 'The Bronx'].reset_index(drop=True)
Bronx_data.head()

In [ ]:
BX_CUISINE_WC = Bronx_data[['Cuisine']]
BX_CUISINE_WC

In [ ]:
BX_CUISINE_WC.to_csv('BX_CUISINE.txt', sep=',', index=False)

In [ ]:
BX_CUISINE_WC = open('BX_CUISINE.txt', 'r').read()

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
# instantiate a word cloud object
BX_CUISINE_NYC = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
BX_CUISINE_NYC.generate(BX_CUISINE_WC)

In [ ]:
# display the word cloud
plt.imshow(BX_CUISINE_NYC, interpolation='bilinear')
plt.axis('off')

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(45)

plt.show()

**Most Preferred Food in The Bronx is -**

1. Italian
2. Puerto Rican
3. Albanian
4. Dominican